# ReAct, a useful program to understand Guillespie algorithm
* hi

* hi
It's not explained yet, but here there are some examples that show how it works more or less

In [1]:
import markdown

# Core functions

* Here there' the code for the three most important functions:

* ReAct: the main function
** Input 
***reactions: The reactions writen as a tuple of size 3xN, where N is an int. Each reaction in the system is defined by 3 consecutive elements of the tuple:
1. Tuple of reactants (R_i,SI_i,R_(i+1),SI_(i+1),...,R_n,SI_n), where n is the number of reactants